**Table of contents**<a id='toc0_'></a>    
- [Localization of Mobile Robot Using Wheel Odometry](#toc1_)    
- [Starting pybullet](#toc2_)    
- [Initial Setup](#toc3_)    
- [Setting Parameters](#toc4_)    
- [Odometry Estimation](#toc5_)    
- [Bonus: Changing Friction](#toc6_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Localization of Mobile Robot Using Wheel Odometry](#toc0_)

(For a manual summarizing the functions available in pybullet, refer to [this link](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf).)

# <a id='toc2_'></a>[Starting pybullet](#toc0_)

In [1]:
import time
import pybullet
import pybullet_data
physics_client = pybullet.connect(pybullet.GUI) 

pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


# <a id='toc3_'></a>[Initial Setup](#toc0_)

We will perform initial setup such as creating the floor, generating the robot, and setting the camera position.

In [2]:
pybullet.resetSimulation() # Reset the simulation space
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # Add path to necessary data for pybullet
pybullet.setGravity(0.0, 0.0, -9.8) # Set gravity as on Earth
time_step = 1./240.
pybullet.setTimeStep(time_step)

# Load the floor
plane_id = pybullet.loadURDF("plane.urdf")

# Load the robot
car_start_pos = [0, 0, 0.1]  # Set the initial position (x, y, z)
car_start_orientation = pybullet.getQuaternionFromEuler([0, 0, 0])  # Set the initial orientation (roll, pitch, yaw)
car_id = pybullet.loadURDF("../urdf/simple_two_wheel_car.urdf", car_start_pos, car_start_orientation)

# Set the camera position and other parameters in GUI mode
camera_distance = 2.0
camera_yaw = 180.0 # deg
camera_pitch = -90.1 # deg
camera_target_position = [0.0, 0.0, 0.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

ven = Mesa
ven = Mesa


In [3]:
import math

class WheelOdometry:
    def __init__(self, initial_pose, wheel_radius, wheel_tread, const_right_wheel, const_left_wheel, initial_angle_right_wheel, initial_angle_left_wheel):
        """
        Class for wheel odometry
        Parameters
        ----------
        initial_pose : tuple
            Initial position and orientation of the robot (x, y, theta)
        wheel_radius : float
            Radius of the wheel [m]
        wheel_tread : float
            Distance between the left and right wheels [m]
        const_right_wheel : float
            Constant for the right wheel (correction value due to friction and environment)
        const_left_wheel : float
            Constant for the left wheel (correction value due to friction and environment)
        initial_angle_right_wheel : float
            Initial angle of the right wheel [rad]
        initial_angle_left_wheel : float
            Initial angle of the left wheel [rad]
        """
        # Constants related to the size of the wheels
        self.WHEEL_DIAMETER = wheel_radius * 2
        self.WHEEL_TREAD = wheel_tread

        # Constants considering the environment and tire material
        self.CONSTANT_RIGHT_WHEEL = const_right_wheel
        self.CONSTANT_LEFT_WHEEL = const_left_wheel

        # Distance per revolution
        self.ONE_REVOLUTION_DISTANCE_RIGHT = math.pi * self.WHEEL_DIAMETER * self.CONSTANT_LEFT_WHEEL
        self.ONE_REVOLUTION_DISTANCE_LEFT = math.pi * self.WHEEL_DIAMETER * self.CONSTANT_RIGHT_WHEEL

        # Initial position and orientation of the robot (x, y, theta)
        self.x = initial_pose[0]
        self.y = initial_pose[1]
        self.theta = initial_pose[2]

        # Previous wheel angles
        self.last_angle_right_wheel = initial_angle_right_wheel
        self.last_angle_left_wheel = initial_angle_left_wheel

    def update_position(self, current_angle_right_wheel, current_angle_left_wheel):
        """
        Update the position and orientation of the robot
        
        Parameters
        ----------
        current_angle_right_wheel : float
            Current angle of the right wheel [rad]
        current_angle_left_wheel : float
            Current angle of the left wheel [rad]
        """
        # Calculate the small change in rotation angle [ΔΘ] of the left and right wheels
        delta_angle_right_wheel = current_angle_right_wheel - self.last_angle_right_wheel
        delta_angle_left_wheel = current_angle_left_wheel - self.last_angle_left_wheel

        # Calculate the number of revolutions from the small change in rotation angle [Δθ] of the left and right wheels (convert 2π → 1 revolution)
        revolution_right_wheel = delta_angle_right_wheel / (2.0 * math.pi)
        revolution_left_wheel = delta_angle_left_wheel / (2.0 * math.pi)

        # Calculate the distance traveled [m] by the left and right wheels
        distance_right_wheel = revolution_right_wheel * self.ONE_REVOLUTION_DISTANCE_RIGHT
        distance_left_wheel = revolution_left_wheel * self.ONE_REVOLUTION_DISTANCE_LEFT

        # Calculate the average travel distance [m] from the left and right travel distances
        distance_avg = (distance_right_wheel + distance_left_wheel) / 2.0

        # Update the position and orientation of the robot
        self.x += distance_avg * math.cos(self.theta)
        self.y += distance_avg * math.sin(self.theta)
        self.theta += math.atan2(distance_right_wheel - distance_left_wheel, self.WHEEL_TREAD)

        # Keep theta within the range -π to π
        if self.theta > math.pi:
            self.theta -= 2 * math.pi
        elif self.theta < -math.pi:
            self.theta += 2 * math.pi

        # Save the current wheel angles
        self.last_angle_right_wheel = current_angle_right_wheel
        self.last_angle_left_wheel = current_angle_left_wheel

    def get_position(self):
        """
        Get the position and orientation of the robot
        """
        return self.x, self.y, self.theta


# <a id='toc4_'></a>[Setting Parameters](#toc0_)

In [4]:
# Indices of the joints for the left and right wheels
RIGHT_WHEEL_IDX = 0
LEFT_WHEEL_IDX = 1

# Constants related to the wheels
WHEEL_RADIUS = 0.05 # Radius of the wheels (match the radius of the wheels in "simple_two_wheel_car.urdf")
WHEEL_THREAD = 0.325 # Distance between the wheels (match the distance between the wheels in "simple_two_wheel_car.urdf")
CONST_RIGHT_WHEEL = 1.0 # Constant for the right wheel (considering friction, material, etc.)
CONST_LEFT_WHEEL = 1.0 # Constant for the left wheel (considering friction, material, etc.)

# Create sliders for linear velocity and angular velocity
linear_vel_slider = pybullet.addUserDebugParameter("linear_velocity", -10, 10, 0)
angular_vel_slider = pybullet.addUserDebugParameter("angular_velocity", -10, 10, 0)

# <a id='toc5_'></a>[Odometry Estimation](#toc0_)

When the simulation is run, you can move the robot by adjusting the sliders on the right to change the speed of the left and right wheels.

Additionally, the "true position of the robot" and the "position estimated by odometry" are displayed on the screen.

In [5]:
import sys

# Initialize debug drawing
pybullet.removeAllUserDebugItems()
replace_pose_texts = []

# Set the robot to the initial position
car_start_x = 0.0
car_start_y = 0.0
car_start_theta = 0.0
car_start_pos = [car_start_x, car_start_y, 0.1]
car_start_orientation = pybullet.getQuaternionFromEuler([0.0, 0.0, car_start_theta])
pybullet.resetBasePositionAndOrientation(car_id, car_start_pos, car_start_orientation)

# Get the initial rotation angles of the left and right wheels
initial_angle_right_wheel = pybullet.getJointState(car_id, RIGHT_WHEEL_IDX)[0]
initial_angle_left_wheel = pybullet.getJointState(car_id, LEFT_WHEEL_IDX)[0]

# Instantiate the odometry class
wheel_odometry = WheelOdometry([car_start_x, car_start_y, car_start_theta], WHEEL_RADIUS, WHEEL_THREAD, CONST_RIGHT_WHEEL, CONST_LEFT_WHEEL, initial_angle_right_wheel, initial_angle_left_wheel)

enable_debug_text = False # Whether to display debug text on the Pybullet GUI screen (note that setting this to True will make the processing heavy)

# Control the mobile robot and calculate odometry based on the values set in the sliders
while True:
    # Get the values from the sliders
    linear_velocity = pybullet.readUserDebugParameter(0)
    angular_velocity = pybullet.readUserDebugParameter(1)

    # Calculate the velocities of the left and right wheels from the linear and angular velocities
    right_wheel_velocity = linear_velocity - angular_velocity * WHEEL_THREAD / 2
    left_wheel_velocity = linear_velocity + angular_velocity * WHEEL_THREAD / 2

    # Set the velocities
    pybullet.setJointMotorControl2(car_id, RIGHT_WHEEL_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=right_wheel_velocity)
    pybullet.setJointMotorControl2(car_id, LEFT_WHEEL_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=left_wheel_velocity)
    pybullet.stepSimulation()

    # Get the current rotation angles of the wheels
    current_angle_wheel_right = pybullet.getJointState(car_id, RIGHT_WHEEL_IDX)[0]
    current_angle_wheel_left = pybullet.getJointState(car_id, LEFT_WHEEL_IDX)[0]

    # Calculate odometry
    wheel_odometry.update_position(current_angle_wheel_right, current_angle_wheel_left)
    odom_x, odom_y, odom_theta = wheel_odometry.get_position()

    # Get the true position and orientation
    true_car_pos, true_car_orientation = pybullet.getBasePositionAndOrientation(car_id)
    true_x = true_car_pos[0]
    true_y = true_car_pos[1]
    true_euler = pybullet.getEulerFromQuaternion(true_car_orientation)
    true_theta = true_euler[2]
    
    # Display the estimated and true positions and orientations on the pybullet screen
    if enable_debug_text:
        if len(replace_pose_texts) == 0:
            replace_pose_texts.append(pybullet.addUserDebugText(f"Estimate Pose x:{odom_x:.2f}, y:{odom_y:.2f}, theta:{math.degrees(odom_theta):.2f}", [-1.0, -0.5, 0.0], textColorRGB=[1,0,0], textSize=1.5))
            replace_pose_texts.append(pybullet.addUserDebugText(f"True Pose x:{true_x:.2f}, y:{true_y:.2f}, theta:{math.degrees(true_theta):.2f}", [-1.0, -0.7, 0.0], textColorRGB=[0,1,0], textSize=1.5))
        else:
            pybullet.addUserDebugText(f"Estimate Pose x:{odom_x:.2f}, y:{odom_y:.2f}, theta:{math.degrees(odom_theta):.2f}", [-1.0, -0.5, 0.0], textColorRGB=[1,0,0], textSize=1.5, replaceItemUniqueId=replace_pose_texts[0])
            pybullet.addUserDebugText(f"True Pose x:{true_x:.2f}, y:{true_y:.2f}, theta:{math.degrees(true_theta)::.2f}", [-1.0, -0.7, 0.0], textColorRGB=[0,1,0], textSize=1.5, replaceItemUniqueId=replace_pose_texts[1])
    
    # Display the same information using print statements
    sys.stdout.write(f"\r【Estimate Pose x:{odom_x:.2f}, y:{odom_y:.2f}, theta:{math.degrees(odom_theta):.2f}】【True Pose x:{true_x:.2f}, y:{true_y:.2f}, theta:{math.degrees(true_theta):.2f}】                       ")

    time.sleep(time_step)

【Estimate Pose x:0.86, y:-0.00, theta:-138.05】【True Pose x:0.88, y:-0.00, theta:-134.19】                       

# <a id='toc6_'></a>[Bonus: Changing Friction](#toc0_)

By running the code below, you can change the friction coefficient of the robot.

After changing the friction coefficient, running the above code again will show that the odometry estimation results change.

In [ ]:
plane_lateral_friction = 1.0 # Friction coefficient of the floor (default is 1.0)
wheel_lateral_friction = 1.0 # Friction coefficient of the wheels (default is 1.0)

# Change the friction of the ground
pybullet.changeDynamics(plane_id, -1, lateralFriction=plane_lateral_friction)

# Change the friction of the robot's wheels
for wheel_link in range(pybullet.getNumJoints(car_id)):
    pybullet.changeDynamics(car_id, wheel_link, lateralFriction=wheel_lateral_friction)